In [1]:
import anndata
import scvelo as scv
import pandas as pd
import os
import numpy as np
import numpy as np
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
import matplotlib.colors as mcolors
from matplotlib.gridspec import GridSpec
import scanpy as sc
SEED = 2024
np.random.seed(SEED)
import unitvelo as utv
from itertools import chain

(Running UniTVelo 0.2.5.2)
2024-12-02 03:57:57


In [2]:
files = ['1_celldancer', '2_scvelo_stochastic', '3_scvelo_dynamical','4_STT','5_unitvelo','6_DeepveloSA', '7_DeepveloGB','8_veloVI',"9_latentvelo","10_Dynamo_stochastic","11_Dynamo_deterministic","12_velocyto",'13_veloVAE','14_veloAE']
#method_names = ['cellDancer', 'scvelo_stochastic', 'scvelo_dynamical', 'UniTvelo', 'velocity','DeepVelo_GB',"veloVI","LatentVelo","Dynamo_stochastic","Dynamo_deterministic","Velocyto"]
simuID_list = ['bifurcating_converging_seed1', 'bifurcating_converging_seed2', 'bifurcating_converging_seed3' ,'bifurcating_cycle_seed1', 'bifurcating_cycle_seed2', 'bifurcating_cycle_seed3' ,'bifurcating_loop_seed1', 'bifurcating_loop_seed2', 'bifurcating_loop_seed3' ,'bifurcating_seed1', 'bifurcating_seed2' ,'bifurcating_seed3' ,'binary_tree_seed1' ,'binary_tree_seed2', 'binary_tree_seed3', 'branching_seed1', 'branching_seed2', 'branching_seed3', 'consecutive_bifurcating_seed1', 'consecutive_bifurcating_seed2' ,'consecutive_bifurcating_seed3', 'converging_seed1', 'converging_seed2','converging_seed3', 'cycle_seed1', 'cycle_seed2', 'cycle_seed3' ,'cycle_simple_seed1' ,'cycle_simple_seed2', 'cycle_simple_seed3', 'disconnected_seed1' ,'disconnected_seed2', 'disconnected_seed3' ,'linear_seed1' ,'linear_seed2', 'linear_seed3', 'linear_simple_seed1' ,'linear_simple_seed2', 'linear_simple_seed3', 'trifurcating_seed1' ,'trifurcating_seed2' ,'trifurcating_seed3' ]


In [3]:
cluster_edges = {"bifurcating_converging_seed1":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sF"),("sD_sE", "sE_sF")],
                "bifurcating_converging_seed2":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sF"),("sD_sE", "sE_sF")],
                "bifurcating_converging_seed3":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sF"),("sD_sE", "sE_sF")],
                "bifurcating_cycle_seed1":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sA"),("sD_sE", "sE_sA"),("sE_sA","sA_sB")],
                "bifurcating_cycle_seed2":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sA"),("sD_sE", "sE_sA"),("sE_sA","sA_sB")],
                "bifurcating_cycle_seed3":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sE"),("sB_sD", "sD_sE"),("sC_sE", "sE_sA"),("sD_sE", "sE_sA"),("sE_sA","sA_sB")],
                "bifurcating_loop_seed1":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sA"),("sC_sA", "sA_sB")],
                "bifurcating_loop_seed2":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sA"),("sC_sA", "sA_sB")],
                "bifurcating_loop_seed3":[("sA_sB", "sB_sC"),("sA_sB", "sB_sD"),("sB_sC", "sC_sA"),("sC_sA", "sA_sB")],
                "bifurcating_seed1":[("sA_sB", "sB_sBmid"),("sB_sBmid", "sBmid_sD"),("sB_sBmid", "sBmid_sC"),("sBmid_sC","sBmid_sD"),("sBmid_sC", "sC_sEndC"),("sBmid_sD", "sD_sEndD")],
                "bifurcating_seed2":[("sA_sB", "sB_sBmid"),("sB_sBmid", "sBmid_sD"),("sB_sBmid", "sBmid_sC"),("sBmid_sC","sBmid_sD"),("sBmid_sC", "sC_sEndC"),("sBmid_sD", "sD_sEndD")],
                "bifurcating_seed3":[("sA_sB", "sB_sBmid"),("sB_sBmid", "sBmid_sD"),("sB_sBmid", "sBmid_sC"),("sBmid_sC","sBmid_sD"),("sBmid_sC", "sC_sEndC"),("sBmid_sD", "sD_sEndD")],
                "binary_tree_seed1":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEmid"),("sE_sEmid","sEmid_sF"),("sEmid_sF","sF_sEndF"),("sE_sEmid","sEmid_sG"),("sEmid_sG","sG_sEndG")],
                "binary_tree_seed2":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEmid"),("sE_sEmid","sEmid_sF"),("sEmid_sF","sF_sEndF"),("sE_sEmid","sEmid_sG"),("sEmid_sG","sG_sEndG")],
                "binary_tree_seed3":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sCmid_sD","sCmid_sE")],
                "branching_seed1":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid","sCmid_sH"),("sCmid_sH","sH_sEndH"),("sC_sCmid","sCmid_sI"),("sCmid_sI","sI_sEndI"),("sC_sCmid","sCmid_sD"),("sCmid_sD","sD_sDmid"),("sD_sDmid","sDmid_sE"),("sDmid_sE","sE_sEndE"),("sD_sDmid","sDmid_sF"),("sDmid_sF","sF_sEndF"),("sD_sDmid","sDmid_sG"),("sDmid_sG","sG_sEndG")],
                "branching_seed2":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid","sCmid_sJ"),("sCmid_sJ","sJ_sEndJ"),("sC_sCmid","sCmid_sI"),("sCmid_sI","sI_sEndI"),("sC_sCmid","sCmid_sD"),("sCmid_sD","sD_sDmid"),("sD_sDmid","sDmid_sE"),("sDmid_sE","sE_sEndE"),("sD_sDmid","sDmid_sF"),("sDmid_sF","sF_sEndF"),("sD_sDmid","sDmid_sG"),("sDmid_sG","sG_sEndG"),("sD_sDmid","sDmid_sH"),("sDmid_sH","sH_sEndH")],
                "branching_seed3":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid","sCmid_sE"),("sCmid_sE","sE_sEndE"),("sC_sCmid","sCmid_sD"),("sCmid_sD","sD_sEndD")],
                "consecutive_bifurcating_seed1":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sDmid"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sD_sDmid","sDmid_sF"),("sDmid_sF","sF_sEndF"),("sD_sDmid","sDmid_sG"),("sDmid_sG","sG_sEndG")],
                "consecutive_bifurcating_seed2":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sDmid"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sD_sDmid","sDmid_sF"),("sDmid_sF","sF_sEndF"),("sD_sDmid","sDmid_sG"),("sDmid_sG","sG_sEndG")],
                "consecutive_bifurcating_seed3":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sDmid"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sD_sDmid","sDmid_sF"),("sDmid_sF","sF_sEndF"),("sD_sDmid","sDmid_sG"),("sDmid_sG","sG_sEndG")],
                "converging_seed1":[("preB_sB", "sB_sD"),("sB_sD","sD_sE"),("preC_sC", "sC_sD"),("sC_sD","sD_sE")],
                "converging_seed2":[("preB_sB", "sB_sD"),("sB_sD","sD_sE"),("preC_sC", "sC_sD"),("sC_sD","sD_sE")],
                "converging_seed3":[("preB_sB", "sB_sD"),("sB_sD","sD_sE"),("preC_sC", "sC_sD"),("sC_sD","sD_sE")],
                "cycle_seed1":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "cycle_seed2":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "cycle_seed3":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "cycle_simple_seed1":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "cycle_simple_seed2":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "cycle_simple_seed3":[("s1_s2","s2_s3"),("s2_s3","s3_s1"),("s3_s1","s1_s2")],
                "disconnected_seed1":[("right_s1_right_s2","right_s2_right_s3"),("right_s2_right_s3","right_s3_right_s1"),("right_s3_right_s1","right_s1_right_s2"),("left_sA_left_sB","left_sB_left_sC"),("left_sB_left_sC","left_sC_left_sEndC")],
                "disconnected_seed2":[("right_s1_right_s2","right_s2_right_s3"),("right_s2_right_s3","right_s3_right_s1"),("right_s3_right_s1","right_s1_right_s2"),("left_sA_left_sB","left_sB_left_sC"),("left_sB_left_sC","left_sC_left_sEndC")],
                "disconnected_seed3":[("right_s1_right_s2","right_s2_right_s3"),("right_s2_right_s3","right_s3_right_s1"),("right_s3_right_s1","right_s1_right_s2"),("left_sA_left_sB","left_sB_left_sC"),("left_sB_left_sC","left_sC_left_sEndC")],
                "linear_seed1":[("sA_sB", "sB_sC"),("sB_sC", "sC_sEndC")],
                "linear_seed2":[("sA_sB", "sB_sC"),("sB_sC", "sC_sEndC")],
                "linear_seed3":[("sA_sB", "sB_sC"),("sB_sC", "sC_sEndC")],
                "linear_simple_seed1":[("1","0")],
                "linear_simple_seed2":[("0","1"),("2","0")],
                "linear_simple_seed3":[("1","0"),("2","1")],
                "trifurcating_seed1":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sC_sCmid","sCmid_sF"),("sCmid_sF","sF_sEndF")],
                "trifurcating_seed2":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sC_sCmid","sCmid_sF"),("sCmid_sF","sF_sEndF")],
                "trifurcating_seed3":[("sA_sB", "sB_sC"),("sB_sC","sC_sCmid"),("sC_sCmid", "sCmid_sD"),("sCmid_sD", "sD_sEndD"),("sC_sCmid", "sCmid_sE"),("sCmid_sE", "sE_sEndE"),("sC_sCmid","sCmid_sF"),("sCmid_sF","sF_sEndF")],
                }

In [4]:
def get_global_coherence(adata,label,basis_embedding,cluster_edges):
    ### read data. Note: the defalut velocity names in layers is 'velocity'.
    #adata = scv.read(adata_dir)
    ### calculate velocity
    #scv.pp.filter_and_normalize(adata, min_shared_counts=0, n_top_genes=2000)
    #scv.pp.moments(adata, n_pcs=30, n_neighbors=30)
    #scv.tl.velocity(adata, mode='stochastic')
    ########## calculate embedding
    scv.tl.velocity_graph(adata)
    scv.tl.velocity_embedding(adata,basis = basis_embedding)
    #################### evaluate
    exp_metrics = utv.evaluate(adata, cluster_edges, label, 'velocity',x_emb = "X_" + basis_embedding)
    ###### global coherence
    in_cluster = exp_metrics["In-cluster Coherence"]
    cross_boundary = exp_metrics["Cross-Boundary Direction Correctness (A->B)"]
    in_res = np.median(list(chain.from_iterable(in_cluster.values())))
    cross_res =  np.median(list(chain.from_iterable(cross_boundary.values())))
    
    cell_number = adata.obs.shape[0]
    
    g1 = []
    for cluster0 in list(in_cluster.keys()):
        data_tmp1 = in_cluster[cluster0]
        p1 = len(data_tmp1)/cell_number
        h1 = np.mean(data_tmp1)
        
        d1 = 0
        count_num = 0
        for edge0 in list(cross_boundary.keys()):
            if cluster0 in edge0:
                ss = cross_boundary[edge0]
                count_num = count_num + 1
                d1 = d1 + np.sign(np.median(ss)) ########## sign
                #d1 <- d1 + np.median(ss) ########## median
        g1.append( p1 * h1 * d1 / count_num)
    return [in_res,cross_res,sum(g1)]

In [5]:
############################## in, cross, global
in_list = {}
cross_list = {}
global_list = {}
label  = "clusters"
basis_embedding = "pca"

for simuID in simuID_list:
    in_list_tmp = {}
    cross_list_tmp = {}
    global_list_tmp = {}
    
    for i in range(len(files)):
    #for i in [4]:
        print("\n")
        print(simuID +"_"+ files[i])
        print("\n")
        file2 = "/lustre/home/zhangxiaochang/project/liyr/data/dyngen/result_0921/velocity/"+simuID+"/"+files[i]+"/"+"anndata.h5ad"
        if os.path.exists(file2):        
            adata = scv.read(file2)
            res  = get_global_coherence(adata,label,basis_embedding,cluster_edges[simuID])
            in_list_tmp[files[i]] = res[0]
            cross_list_tmp[files[i]] = res[1]
            global_list_tmp[files[i]] = res[2]
        else:
            in_list_tmp[files[i]] = np.nan
            cross_list_tmp[files[i]] = np.nan
            global_list_tmp[files[i]] = np.nan
    in_list[simuID] = in_list_tmp
    cross_list[simuID] = cross_list_tmp
    global_list[simuID] = global_list_tmp



bifurcating_converging_seed1_1_celldancer


computing velocity graph (using 1/40 cores)


  0%|          | 0/2500 [00:00<?, ?cells/s]

    finished (0:00:03) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:01) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('sA_sB', 'sB_sC'): -0.3044284377122978, ('sA_sB', 'sB_sD'): -0.165665197923096, ('sB_sC', 'sC_sE'): -0.26542912615580755, ('sB_sD', 'sD_sE'): -0.09390911004875634, ('sC_sE', 'sE_sF'): 0.008193407479629761, ('sD_sE', 'sE_sF'): -0.13759413459924866}
Total Mean: -0.15980543315992943
# In-cluster Coherence
{'sA_sB': 0.8331330472874554, 'sB_sC': 0.8182980280201699, 'sB_sD': 0.8217447324495714, 'sC_sE': 0.815559685549247, 'sD_sE': 0.8171164470349467, 'sE_sF': 0.8223788336045686}
Total Mean: 0.8213717956576598


bifurcating_converging_seed1_2_scvelo_stochastic


computing velocity graph (using 1/40 cores)


  0%|          | 0/2500 [00:00<?, ?cells/s]

    finished (0:00:02) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:01) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('sA_sB', 'sB_sC'): 0.1787221762251881, ('sA_sB', 'sB_sD'): 0.22121023491867792, ('sB_sC', 'sC_sE'): -0.038500023798700336, ('sB_sD', 'sD_sE'): 0.1789090032626677, ('sC_sE', 'sE_sF'): 0.2410786285871981, ('sD_sE', 'sE_sF'): 0.25222300121141644}
Total Mean: 0.172273836734408
# In-cluster Coherence
{'sA_sB': 0.8832845, 'sB_sC': 0.78726757, 'sB_sD': 0.70358014, 'sC_sE': 0.8369485, 'sD_sE': 0.6927322, 'sE_sF': 0.68126297}
Total Mean: 0.7641792893409729


bifurcating_converging_seed1_3_scvelo_dynamical


computing velocity graph (using 1/40 cores)


...

    finished (0:00:02) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('sA_sB', 'sB_sC'): 0.11800647397654657, ('sB_sC', 'sC_sCmid'): 0.27022471243594676, ('sC_sCmid', 'sCmid_sD'): 0.1706323961420611, ('sCmid_sD', 'sD_sEndD'): 0.13668221080190307, ('sC_sCmid', 'sCmid_sE'): 0.16712954206991062, ('sCmid_sE', 'sE_sEndE'): 0.13069272957701702, ('sC_sCmid', 'sCmid_sF'): 0.11135750460361057, ('sCmid_sF', 'sF_sEndF'): 0.013367540219230799}
Total Mean: 0.13976163872827832
# In-cluster Coherence
{'sA_sB': 0.8352691743674094, 'sB_sC': 0.898449923592231, 'sC_sCmid': 0.9215820321263458, 'sCmid_sD': 0.8980994826674258, 'sCmid_sE': 0.8988641727726973, 'sCmid_sF': 0.9003868179183736, 'sD_sEndD': 0.8926582960006746, 'sE_sEndE': 0.8418504065666794, 'sF_sEndF': 0.8764513952267335}
Total Mean: 0.8848457

  0%|          | 0/2448 [00:00<?, ?cells/s]

    finished (0:00:02) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('sA_sB', 'sB_sC'): 0.20738222237314427, ('sB_sC', 'sC_sCmid'): 0.25715171921944124, ('sC_sCmid', 'sCmid_sD'): -0.13375966402174527, ('sCmid_sD', 'sD_sEndD'): 0.22860784259041383, ('sC_sCmid', 'sCmid_sE'): -0.16616471614429548, ('sCmid_sE', 'sE_sEndE'): 0.42405694274091327, ('sC_sCmid', 'sCmid_sF'): -0.2520621935285233, ('sCmid_sF', 'sF_sEndF'): 0.5007828389447414}
Total Mean: 0.13324937402176124
# In-cluster Coherence
{'sA_sB': 0.9984195950268083, 'sB_sC': 0.9928084508791973, 'sC_sCmid': 0.9942668622282291, 'sCmid_sD': 0.9506326961935812, 'sCmid_sE': 0.9451968955437188, 'sCmid_sF': 0.9588163423412618, 'sD_sEndD': 0.9950993146283139, 'sE_sEndE': 0.8277147465129923, 'sF_sEndF': 0.7861824221606513}
Total Mean: 0.93879

  0%|          | 0/2500 [00:00<?, ?cells/s]

    finished (0:00:02) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)
computing velocity embedding
    finished (0:00:00) --> added
    'velocity_pca', embedded velocity vectors (adata.obsm)
# Cross-Boundary Direction Correctness (A->B)
{('sA_sB', 'sB_sC'): -0.03725089707308793, ('sB_sC', 'sC_sCmid'): -0.22982935709916733, ('sC_sCmid', 'sCmid_sD'): -0.370246897903457, ('sCmid_sD', 'sD_sEndD'): -0.3216118556116057, ('sC_sCmid', 'sCmid_sE'): -0.40257797511231, ('sCmid_sE', 'sE_sEndE'): -0.4421286280498706, ('sC_sCmid', 'sCmid_sF'): -0.24873821847215224, ('sCmid_sF', 'sF_sEndF'): -0.33068098539479873}
Total Mean: -0.2978831018395562
# In-cluster Coherence
{'sA_sB': 0.9601322, 'sB_sC': 0.9230661, 'sC_sCmid': 0.93809354, 'sCmid_sD': 0.8927124, 'sCmid_sE': 0.9200453, 'sCmid_sF': 0.89590466, 'sD_sEndD': 0.93322885, 'sE_sEndE': 0.94277704, 'sF_sEndF': 0.94324577}
Total Mean: 0.9276895523071289


In [6]:
pd.DataFrame(in_list).to_csv("eval/eval_incluster.csv")
pd.DataFrame(cross_list).to_csv("eval/eval_direction.csv")
pd.DataFrame(global_list).to_csv("eval/eval_global.csv")